### Imports

In [1]:
import modin.pandas as md
import json
import redis
import os

### Load CSV Data

In [2]:
df = md.read_csv('../data/tmdbDetails.csv')
df.drop_duplicates(inplace=True, keep='last')

2024-09-03 07:51:45,731	INFO worker.py:1781 -- Started a local Ray instance.
Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


### Load Redis Data

In [3]:
# Retrieve all Redis keys & values
r = redis.Redis(
    host='127.0.0.1',
    port=6379,
    charset="utf-8",
    decode_responses=True,
    db=0
)

redisKeys = r.keys('*')
redisValues = r.mget(redisKeys)

redisJson = []

for i, v in enumerate(redisValues):
    try:
        url = redisKeys[i]
        j = json.loads(v)
        redisJson.append(j)
    except: # We had some test keys in the db that are not json
        pass

In [4]:
redisDf = md.DataFrame.from_dict(redisJson)
fullDf  = md.concat([df, redisDf], axis=0)

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [5]:
fullDf.drop(columns = [c for c in fullDf.columns if 'Unnamed' in c], inplace = True)
print(f"LENGTH OF DF: {len(fullDf)}")

LENGTH OF DF: 1149980


### Output to csv

In [6]:
fullDf.to_csv('../data/tmdbDetails.csv')